<a href="https://colab.research.google.com/github/dreamproit/annif-bill-committees-us/blob/BML-26-investigate-annif-tool-for-classification-bill-committees-us-dataset/spacy_multi_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install  spaCy datasets pandas pandarallel annif
# ! pip install dask cudf

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16673 sha256=a3b969be315ad8250d390d39b45c3bc943477827941584d51b91adc8633c5e00
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

dataset = load_dataset("dreamproit/bill_labels_us")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
pds = dataset['train'].to_pandas()

In [ ]:
import pandas as pd

TARGET_NUM_OF_ITEMS = 30

df = pds
policy_areas_counts = df['policy_area'].value_counts()
policy_areas_ordered = policy_areas_counts.index.tolist()

sampled_dfs = []

def create_policy_area_df(df, num_of_items: int, policy_area_name: str):
    policy_area_df = df[df['policy_area'] == policy_area_name].copy()

    # Define the ranges
    ranges = sorted([0, 1000, 2000, 5000, 10_000, 25_000, 50_000, 75_000, 100_000, 150_000, 250_000, 350_000, 500_000, 650_000, 800_000, policy_area_df['text_length'].max()])

    # Create the buckets
    policy_area_df['bucket'] = pd.cut(policy_area_df['text_length'], bins=ranges, include_lowest=True)

    # Sample items from the entire DataFrame without replacement
    subset_df = policy_area_df.sample(n=min(num_of_items, len(policy_area_df)), replace=False, random_state=1)

    # Convert lists in 'legislative_subjects' to tuples to avoid 'unhashable type: 'list'' error
    subset_df['legislative_subjects'] = subset_df['legislative_subjects'].apply(tuple)

    # Reset the index of the subset DataFrame
    subset_df.reset_index(drop=True, inplace=True)

    return subset_df

for policy_area_name in policy_areas_ordered:
    sampled_df = create_policy_area_df(df, num_of_items=TARGET_NUM_OF_ITEMS, policy_area_name=policy_area_name)
    print(f'Policy area: "{policy_area_name}" sample df shape: {sampled_df.shape}')
    # duplicates = sampled_df.duplicated()
    # print(f"Number of duplicate rows = {duplicates.sum()}")
    sampled_df.info()
    # print(f'Policy area: "{policy_area_name}" sample df info: {subset_df.info()}')
    total_chars = sampled_df['text_length'].sum()
    print(f'Policy area: "{policy_area_name}" sample df total_chars: {total_chars}')
    # plot_bucket_distribution(sampled_df, policy_area_name, 'bucket')
    # plot_field_distribution(sampled_df, policy_area_name, 'congress')
    # plot_field_distribution(sampled_df, policy_area_name, 'bill_type')
    # plot_field_distribution(sampled_df, policy_area_name, 'bill_version')
    sampled_dfs.append(sampled_df)

combined_df = pd.concat(sampled_dfs)
print(f'Train df for "{TARGET_NUM_OF_ITEMS}" rows have shape: "{combined_df.shape}"')

Policy area: "Health" sample df shape: (30, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    30 non-null     object  
 1   congress              30 non-null     int64   
 2   bill_type             30 non-null     object  
 3   bill_number           30 non-null     int64   
 4   bill_version          30 non-null     object  
 5   title                 30 non-null     object  
 6   sections              30 non-null     object  
 7   sections_length       30 non-null     int64   
 8   text                  30 non-null     object  
 9   text_length           30 non-null     int64   
 10  policy_area           30 non-null     object  
 11  legislative_subjects  30 non-null     object  
 12  bucket                30 non-null     category
dtypes: category(1), int64(4), object(8)
memory usage: 3.7+ KB
Policy

In [ ]:
from sklearn.utils import shuffle


test_data = []
train_data = []
validate_data = []

for group, items in combined_df.groupby(['policy_area', 'bucket']):
  print(group, len(items))
  train = shuffle(items).sample(frac=0.7)
  train_data.append(train)
  # print("Add train:", len(train))

  df_rest = items.loc[~items.index.isin(train.index)]

  test = df_rest.sample(frac=0.5)
  test_data.append(df_rest)
  # print("Add test:", len(test))

  # validate = df_rest.loc[~df_rest.index.isin(test.index)]
  # validate_data.append(validate)


  # print("Add validate:", len(validate))

  # print(set(test.index).intersection(validate.index))
  # print(set(validate.index).intersection(train.index))
  # print(set(test.index).intersection(train.index))

test = pd.concat(test_data)
train = pd.concat(train_data)
# validate = pd.concat(validate_data)

print("Test data: ", len(test))
print("Train data: ", len(train))
# print("Validate data: ", len(train))

# print(set(test.index).intersection(validate.index))

print("="*20, "TRAIN", train.shape, "="*20, "\n", train.head())
print("="*20, "TEST",  test.shape, "="*20, "\n", test.head())
# print("="*20, "VALIDATE\n",  validate.shape, "="*20, "\n", validate.head())



('Agriculture and Food', Interval(-0.001, 1000.0, closed='right')) 9
('Agriculture and Food', Interval(1000.0, 2000.0, closed='right')) 6
('Agriculture and Food', Interval(2000.0, 5000.0, closed='right')) 4
('Agriculture and Food', Interval(5000.0, 10000.0, closed='right')) 3
('Agriculture and Food', Interval(10000.0, 25000.0, closed='right')) 4
('Agriculture and Food', Interval(25000.0, 50000.0, closed='right')) 2
('Agriculture and Food', Interval(50000.0, 75000.0, closed='right')) 1
('Agriculture and Food', Interval(150000.0, 250000.0, closed='right')) 1
('Animals', Interval(-0.001, 1000.0, closed='right')) 10
('Animals', Interval(1000.0, 2000.0, closed='right')) 7
('Animals', Interval(2000.0, 5000.0, closed='right')) 7
('Animals', Interval(5000.0, 10000.0, closed='right')) 3
('Animals', Interval(10000.0, 25000.0, closed='right')) 3
('Armed Forces and National Security', Interval(-0.001, 1000.0, closed='right')) 9
('Armed Forces and National Security', Interval(1000.0, 2000.0, closed

In [ ]:
import pickle
import os

if not os.path('/content/drive/MyDrive/DATASET/test.pkl').exists():
  pickle.dump(test, open('/content/drive/MyDrive/DATASET/test.pkl', 'wb'))
if not os.path('/content/drive/MyDrive/DATASET/train.pkl').exists():
  pickle.dump(train, open('/content/drive/MyDrive/DATASET/train.pkl', 'wb'))
if not os.path('/content/drive/MyDrive/DATASET/df.pkl').exists():
  pickle.dump(df, open('/content/drive/MyDrive/DATASET/df.pkl', 'wb'))

In [ ]:
import pickle

test = pickle.load(open('/content/drive/MyDrive/DATASET/test.pkl', 'rb'))
train = pickle.load(open('/content/drive/MyDrive/DATASET/train.pkl', 'rb'))
df = pickle.load(open('/content/drive/MyDrive/DATASET/df.pkl', 'rb'))


In [ ]:
len(df)

119569

In [ ]:
policy_areas_counts = df['policy_area'].value_counts()
policy_areas_ordered = policy_areas_counts.index.tolist()

policy_areas_ordered

['Health',
 'Taxation',
 'Armed Forces and National Security',
 'International Affairs',
 'Government Operations and Politics',
 'Foreign Trade and International Finance',
 'Public Lands and Natural Resources',
 'Crime and Law Enforcement',
 'Congress',
 'Education',
 'Transportation and Public Works',
 'Finance and Financial Sector',
 'Commemorations',
 'Energy',
 'Commerce',
 'Immigration',
 'Environmental Protection',
 'Labor and Employment',
 'Science, Technology, Communications',
 'Agriculture and Food',
 'Economics and Public Finance',
 'Emergency Management',
 'Social Welfare',
 'Housing and Community Development',
 'Native Americans',
 'Civil Rights and Liberties, Minority Issues',
 'Law',
 'Water Resources Development',
 'Families',
 'Sports and Recreation',
 'Animals',
 'Arts, Culture, Religion',
 'Private Legislation',
 'Social Sciences and History']

In [ ]:
from spacy.tokens import DocBin, Doc
from tqdm import tqdm
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.max_length=10_000_000
nlp(test['text'].iloc[0])

def transform_text_to_docs(db):
  def f(l):
    # nlp = spacy.load("en_core_web_sm")
    # nlp.max_length=2_000_000
    if isinstance(l['text'], (str, bytes)):
      doc = nlp(l['text'])
      doc.cats = {
          category: 1 if category in l['legislative_subjects'] else 0
          for category in policy_areas_ordered
      }
      db.add(doc)
      return l
  return f

# train.apply(transform_text_to_docs , axis=1)
# nlp(train['text'][0])

In [ ]:
import psutil

psutil.cpu_count(logical=False)

4

In [ ]:
# import cudf
import dask.dataframe as dd
from dask.multiprocessing import get
from tqdm import tqdm

from pandarallel import pandarallel
from dask.diagnostics import ProgressBar
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from dask.diagnostics import visualize

pandarallel.initialize(progress_bar=True, use_memory_fs=True)

# tt = test[['id', 'text', 'legislative_subjects']]
# test_gf = cudf.from_pandas(tt)  # convert pandas partitions into cudf partitions
# test_gf.apply(transform_text_to_docs)

for _df, file_name in [
    # (test[['id', 'text', 'legislative_subjects', 'text_length']], '/content/drive/MyDrive/DATASET/test_db.pkl'),
    (train[['id', 'text', 'legislative_subjects', 'text_length']], '/content/drive/MyDrive/DATASET/train_db.pkl'),
]:
  # docs_df = _df[:1000].parallel_apply(transform_text_to_docs, axis=1)
  # pickle.dump(docs_df, open(file_name, 'wb'))
  with (
      ProgressBar(),
      # Profiler() as prof,
      # ResourceProfiler(dt=0.25) as rprof,
      # CacheProfiler() as cprof
  ):
    db = DocBin()
    _df = _df.sort_values('text_length', ascending=False)
    test_dd = dd.from_pandas(
      _df, npartitions=round(len(_df) / 30) or 1
    )
    test_dd = test_dd.map_partitions(
        lambda df: df.apply(transform_text_to_docs(db), axis=1)
    ).compute(scheduler='threads')
    # test_dd.to_parquet(file_name)
    db.to_disk(file_name)



# test_dd.apply(transform_text_to_docs, axis=1).compute()
# test = test.map_partitions(cudf.from_pandas)  # convert pandas partitions into cudf partitions
# train = train.map_partitions(cudf.from_pandas)  # convert pandas partitions into cudf partitions


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
[########################################] | 100% Completed | 256.99 s


In [ ]:
from tqdm import tqdm
from spacy.tokens import DocBin, Doc


if 0:
  for _df, file_name in [
      # (test[['id', 'text', 'legislative_subjects']][:1000], '/content/drive/MyDrive/DATASET/test_db.pkl'),
      (train[['id', 'text', 'legislative_subjects', 'text_length']], '/content/drive/MyDrive/DATASET/train_db.pkl'),
  ]:
    db = DocBin()

    docs = nlp.pipe(_df.sort_values('text_length')['text'].values.tolist(), disable=["ner", "parser"])
    cats = _df['legislative_subjects'].values.tolist()
    labels = policy_areas_ordered

    for doc, cat in tqdm(zip(docs, cats), total=len(cats)):
      for l in labels:
          if l in cat:
              doc.cats[l] = 1
          else:
              doc.cats[l] = 0
      db.add(doc)
    db.to_disk(file_name)


 69%|██████▉   | 15104/21803 [07:23<03:16, 34.09it/s]


KeyboardInterrupt: 

In [ ]:
[{k: v for k, v in i.items() if v} for i in db.cats[:10]]

[{},
 {'Congress': 1},
 {'Armed Forces and National Security': 1,
  'Government Operations and Politics': 1,
  'Congress': 1,
  'Economics and Public Finance': 1},
 {},
 {'Immigration': 1},
 {},
 {'Foreign Trade and International Finance': 1},
 {'Congress': 1, 'Immigration': 1},
 {},
 {}]

In [ ]:
print(len(test))
print(len(train))

283
737
